In [ ]:
import asyncio
import json

from binance import AsyncClient, DepthCacheManager, BinanceSocketManager

async def main():

    # initialise the client
    client = await AsyncClient.create()

    # run some simple requests
    print(json.dumps(await client.get_exchange_info(), indent=2))

    print(json.dumps(await client.get_symbol_ticker(symbol="BTCUSDT"), indent=2))

    # initialise websocket factory manager
    bsm = BinanceSocketManager(client)

    # create listener using async with
    # this will exit and close the connection after 5 messages
    async with bsm.trade_socket('ETHBTC') as ts:
        for _ in range(5):
            res = await ts.recv()
            print(f'recv {res}')

    # get historical kline data from any date range

    # fetch 1 minute klines for the last day up until now
    klines = client.get_historical_klines("BNBBTC", AsyncClient.KLINE_INTERVAL_1MINUTE, "1 day ago UTC")

    # use generator to fetch 1 minute klines for the last day up until now
    async for kline in await client.get_historical_klines_generator("BNBBTC", AsyncClient.KLINE_INTERVAL_1MINUTE, "1 day ago UTC"):
        print(kline)

    # fetch 30 minute klines for the last month of 2017
    klines = client.get_historical_klines("ETHBTC", Client.KLINE_INTERVAL_30MINUTE, "1 Dec, 2017", "1 Jan, 2018")

    # fetch weekly klines since it listed
    klines = client.get_historical_klines("NEOBTC", Client.KLINE_INTERVAL_1WEEK, "1 Jan, 2017")

    # setup an async context the Depth Cache and exit after 5 messages
    async with DepthCacheManager(client, symbol='ETHBTC') as dcm_socket:
        for _ in range(5):
            depth_cache = await dcm_socket.recv()
            print(f"symbol {depth_cache.symbol} updated:{depth_cache.update_time}")
            print("Top 5 asks:")
            print(depth_cache.get_asks()[:5])
            print("Top 5 bids:")
            print(depth_cache.get_bids()[:5])

    # Vanilla options Depth Cache works the same, update the symbol to a current one
    options_symbol = 'BTC-210430-36000-C'
    async with OptionsDepthCacheManager(client, symbol=options_symbol) as dcm_socket:
        for _ in range(5):
            depth_cache = await dcm_socket.recv()
            count += 1
            print(f"symbol {depth_cache.symbol} updated:{depth_cache.update_time}")
            print("Top 5 asks:")
            print(depth_cache.get_asks()[:5])
            print("Top 5 bids:")
            print(depth_cache.get_bids()[:5])

    await client.close_connection()

if __name__ == "__main__":

    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())

In [1]:
import asyncio
import json

from binance import AsyncClient

In [40]:
df.index.dtype

datetime64[ns, UTC]

In [16]:
df

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2024-07-10 00:00:00+00:00,58033.882812,58033.882812,58033.882812,58033.882812,58033.882812,0
2024-07-10 00:02:00+00:00,58009.136719,58009.136719,58009.136719,58009.136719,58009.136719,0
2024-07-10 00:03:00+00:00,58009.136719,58009.136719,58009.136719,58009.136719,58009.136719,0
2024-07-10 00:05:00+00:00,57926.828125,57926.828125,57926.828125,57926.828125,57926.828125,57344
2024-07-10 00:06:00+00:00,57902.316406,57902.316406,57902.316406,57902.316406,57902.316406,0
...,...,...,...,...,...,...
2024-07-10 03:20:00+00:00,57899.914062,57899.914062,57899.914062,57899.914062,57899.914062,0
2024-07-10 03:21:00+00:00,57914.246094,57914.246094,57914.246094,57914.246094,57914.246094,0
2024-07-10 03:22:00+00:00,57916.332031,57916.332031,57916.332031,57916.332031,57916.332031,325632


In [3]:
import pygame
import pymunk
from pymunk.pygame_util import DrawOptions

# Initialize pygame and pymunk
pygame.init()
screen = pygame.display.set_mode((800, 600))
clock = pygame.time.Clock()
space = pymunk.Space()
space.gravity = (0, 900)  # Gravity (x, y)

# Load the character image
image_path = "/home/nitin/Downloads/istockphoto-628529954-2048x2048.jpg"
character_image = pygame.image.load(image_path)
character_image = pygame.transform.scale(character_image, (100, 100))
character_rect = character_image.get_rect(center=(400, 100))

# Create a pymunk body and shape for the character
body = pymunk.Body(1, pymunk.inf)
body.position = (400, 100)
shape = pymunk.Poly.create_box(body, (100, 100))
shape.elasticity = 0.5
space.add(body, shape)

# Create the ground
ground = pymunk.Segment(space.static_body, (0, 590), (800, 590), 5)
ground.elasticity = 0.5
space.add(ground)

# Draw options
draw_options = DrawOptions(screen)

# Main simulation loop
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Update physics
    space.step(1/60.0)

    # Clear screen
    screen.fill((255, 255, 255))

    # Draw character
    screen.blit(character_image, (body.position.x - 50, body.position.y - 50))

    # Draw ground
    space.debug_draw(draw_options)

    # Update display
    pygame.display.flip()
    clock.tick(60)

pygame.quit()


AttributeError: module 'pymunk' has no attribute 'inf'